### WBS Coding School
___
# Data Engineering Project

This is a data engineering project, in which I use Python, MySQL and AWS Services to create and automatically update an online database.

It is a learning project, in which I practise several data engineering techniques, such as API calls and AWS Lambda functions.

Our tasks are:
- [x] Collect data
- [x] Clean data
- [x] Create a database
- [ ] Update the database with the latest data
- [ ] Move the data pipeline to the Cloud (AWS)

___

# Create a Database
This script creates a MySQL database on an AWS RDS instance, creates all its tables and populates the static ones (`cities`, `airports`, `cities_airports`, `populations`).

The package `pymysql` is used for running SQL queries to create the database and its tables.

Note that previous to running this script, a Cloud MySQL database on an AWS RDS instance needs to be set up.

### Table of contents:
- [Connect to the Cloud MySQL instance](#connect)
- [Create database & static tables](#create_database)
- [Fill static tables](#fill_tables)

#### Import Libraries & Dataframes

In [6]:
import pandas as pd
import pymysql
import config_file

In [4]:
cities_df_sql = pd.read_csv("dataframes/cleaned/cities_df_clean.csv")
populations_df_sql = pd.read_csv("dataframes/cleaned/populations_df_clean.csv")
airports_df_sql = pd.read_csv("dataframes/cleaned/airports_df_clean.csv")
cities_airports_df_sql = pd.read_csv("dataframes/cleaned/cities_airports_df_clean.csv")
# These are all dataframes that will become static tables, 
# as the information they contain won't change very often.

<a id="connect"></a>
#### Connect to the Cloud MySQL instance
(set up a MySQL database on an AWS RDS instance beforehand)

In [5]:
# The connection information is contained in the config_file.py
user = config_file.AWS_DATABASE_USER
password = config_file.AWS_DATABASE_PASSWORD
host = config_file.AWS_DATABASE_HOST
port = config_file.AWS_DATABASE_PORT
schema = config_file.AWS_DATABASE_SCHEMA

<a id="create_database"></a>
## Create database & static tables

In [4]:
# Create a list of SQL queries for creating the database and its empty tables:

sql_queries_create_database = [
    'CREATE DATABASE IF NOT EXISTS gans;',
    
    'USE gans;',
    
    '''
    CREATE TABLE cities (
        city_id INT AUTO_INCREMENT,
        city_name VARCHAR(255),
        country VARCHAR(255),
        latitude DECIMAL(8,5),
        longitude DECIMAL(8,5),
        altitude INT,
        PRIMARY KEY (city_id)
    );
    ''',
    
    '''
    CREATE TABLE populations (
        city_id INT,
        city_name VARCHAR(255),
        population INT,
        PRIMARY KEY (city_id, population),
        FOREIGN KEY (city_id) REFERENCES cities(city_id)
    );
    ''',
    
    '''
    CREATE TABLE weather (
        weather_id INT AUTO_INCREMENT,
        city_id INT,
        forecast_time DATETIME,
        outlook VARCHAR(255),
        outlook_description VARCHAR(255),
        temperature DECIMAL(4,2),
        feels_like DECIMAL(4,2),
        wind_speed DECIMAL(4,2),
        PRIMARY KEY (weather_id),
        FOREIGN KEY (city_id) REFERENCES cities(city_id)
    );
    ''',

    '''
    CREATE TABLE airports (
        airport_icao VARCHAR(4),
        airport_name VARCHAR(255),
        latitude DECIMAL,
        longitude DECIMAL,
        PRIMARY KEY (airport_icao)
    );
    ''',

    '''
    CREATE TABLE IF NOT EXISTS cities_airports (
        city_id INT NOT NULL,
        airport_icao VARCHAR(4) NOT NULL,
        city_name VARCHAR(255),
        airport_name VARCHAR(255),
        FOREIGN KEY (city_id) REFERENCES cities(city_id),
        FOREIGN KEY (airport_icao) REFERENCES airports(airport_icao)
    );''',

    '''
    CREATE TABLE arrivals (
        flight_id INT NOT NULL AUTO_INCREMENT,
        flight_number VARCHAR (255),
        arrival_icao VARCHAR(4),
        arrival_time DATETIME,
        departure_icao VARCHAR(4),
        PRIMARY KEY (flight_id),
        FOREIGN KEY (arrival_icao) REFERENCES airports (airport_icao)
    );
    '''
]

#### Run the SQL queries

In [5]:
# Connect to the MySQL server and create a cursor
conn = pymysql.connect(host=host, user=user, password=password)
cursor = conn.cursor()

# Execture the queries
for query in sql_queries_create_database:
    cursor.execute(query)

# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()


<a id="fill_tables"></a>
## Fill static tables
(`cities`, `populations`, `airports`, `cities_airports`)

In [7]:
# Create a connection to the MySQL database
conn = pymysql.connect(host=host, user=user, password=password, database=schema)

# Create a cursor
cursor = conn.cursor()

# Insert data from the cities DataFrame into the cities table
sql_query_cities = """
INSERT INTO cities (city_name, country, latitude, longitude, altitude)
VALUES (%s, %s, %s, %s, %s)
"""
cities_tuples = list(cities_df_sql.itertuples(index=False, name=None))
cursor.executemany(query=sql_query_cities, args=cities_tuples)

# Fill populations table
sql_query_populations = """
INSERT INTO populations (city_id, city_name, population)
VALUES (%s, %s, %s)
"""
populations_tuples = list(populations_df_sql.itertuples(index=False, name=None))
cursor.executemany(query=sql_query_populations, args=populations_tuples)

# Fill airports table
sql_query_airports = """
INSERT INTO airports (airport_icao, airport_name, latitude, longitude)
VALUES (%s, %s, %s, %s)
"""
airports_tuples = list(airports_df_sql.itertuples(index=False, name=None))
cursor.executemany(query=sql_query_airports, args=airports_tuples)

# Fill cities_airports table
sql_query_cities_airports = """
INSERT INTO cities_airports (airport_icao, city_id, city_name, airport_name)
VALUES (%s, %s, %s, %s)
"""
cities_airports_tuples = list(cities_airports_df_sql.itertuples(index=False, name=None))
cursor.executemany(query=sql_query_cities_airports, args=cities_airports_tuples)

# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()


#### Test whether the tables are populated

In [8]:
# Connect to the MySQL server and create a cursor
conn = pymysql.connect(host=host, user=user, password=password, database=schema)
cursor = conn.cursor()

# Execture the query
cursor.execute("SELECT * FROM airports")

# Commit the changes to the database
conn.commit()

# Get the results:
results = cursor.fetchall()

# Close the cursor and connection
cursor.close()
conn.close()


# Print the query results
results

(('EDDB', 'Berlin Brandenburg', Decimal('52'), Decimal('13')),
 ('EDDC', 'Dresden ', Decimal('51'), Decimal('14')),
 ('EGGW', 'London Luton', Decimal('52'), Decimal('0')),
 ('EGKK', 'London Gatwick', Decimal('51'), Decimal('0')),
 ('EGKR', 'Redhill Aerodrome', Decimal('51'), Decimal('0')),
 ('EGLC', 'London City', Decimal('52'), Decimal('0')),
 ('EGLL', 'London Heathrow', Decimal('51'), Decimal('0')),
 ('EGSS', 'London Stansted', Decimal('52'), Decimal('0')),
 ('LEMD', 'Madrid Adolfo Suárez –Barajas', Decimal('40'), Decimal('-4')),
 ('RJTT', 'Tokyo ', Decimal('36'), Decimal('140')),
 ('ZSPD', 'Shanghai Pudong', Decimal('31'), Decimal('122')),
 ('ZSSS', 'Shanghai Hongqiao', Decimal('31'), Decimal('121')))